<h1>Spacetime Hot Spot Analysis - Australian Bushfire Demo (v0.1.0)</h1>

<h2>Part 1: Load Dependencies and Data</h2>

<h3>Dependencies</h3>

The first step is to load the dependencies required to prepare the data, perform the analysis, and visualise the results. Pandas and folium are both external open-source libraries used for manipulating data and mapping geographical data, respectively. GeoJikuu is an open-source library for analysing spatiotemporal data and hence will be used for the spacetime hot spot analysis as well as any required pre-processing steps.

In [1]:
# External
import pandas as pd
import folium
import warnings
warnings.filterwarnings("ignore")

# GeoJikuu
from preprocessing.projection import *
from aggregation.point_aggregators import *
from hypothesis_testing.hot_spot_analysis import *
from preprocessing.conversion_tools import DateConvertor
#from preprocessing.normalisation import MinMaxScaler

<h3>Data</h3>

The next step is to load the dataset to which the hot spot analysis will be applied. For this demo, we will be using a dataset titled: [19th Century Australian Bushfire Reporting](https://ghap.tlcmap.org/publicdatasets/170), courtesy of Dr. Fiannuala Morgan.

This dataset contains the approximate locations of bushfires in Australia from 1824 to 1899. As per the dataset's description, these locations were extracted from newspapers and geo-located using GHAP.

Let's start by loading the dataset, and then examining the first five entries.

In [2]:
df = pd.read_csv("bushfiredemo.csv")
df.head()

,ghap_id,dataset_id,title,recordtype_id,latitude,longitude,datestart,dateend,placename,created_at,updated_at,State,Newspaper Place of Publication,Newspaper,Article Word Count,Article Link,coordinates
0,t79e4,170,"TRIBUTARY LINES, Addressed to LIEUTENANT GOVER...",1,-33.906667,151.059444,1824-05-14,1824-05-14,GREECE,23/01/2022 13:51,23/01/2022 13:51,NSW,Hobart,Hobart Town Gazette and Van Diemen's Land Adve...,276,https://nla.gov.au/nla.news-article1090181,"-33.90666667,151.0594444"
1,t79e5,170,IMPROVED TRAVELLING.,1,-33.751528,150.994167,1831-12-24,1831-12-24,BAULKHAM HILLS,23/01/2022 13:51,23/01/2022 13:51,NSW,Sydney,The Sydney Gazette and New South Wales Adverti...,1280,https://nla.gov.au/nla.news-article2204168,"-33.75152778,150.9941667"
2,t79e6,170,FIRE.,1,-32.055085,115.745900,1834-03-15,1834-03-15,FREMANTLE,23/01/2022 13:51,23/01/2022 13:51,WA,Perth,The Perth Gazette and Western Australian Journ...,289,https://nla.gov.au/nla.news-article641603,"-32.055085,115.7459"
3,t79e7,170,The Broad and Liberal.,1,-19.225000,138.350000,1835-02-14,1835-02-14,NORFOLK,23/01/2022 13:51,23/01/2022 13:51,QLD,Hobart,The True Colonist Van Diemen's Land Political ...,2493,https://nla.gov.au/nla.news-article200328486,"-19.225,138.35"
4,t79e9,170,1835. SKETCH OF OCCURRENCES DURING THE YEAR.,1,-33.641667,151.267778,1835-12-19,1835-12-19,ELIZA BAY,23/01/2022 13:51,23/01/2022 13:51,NSW,Perth,The Perth Gazette and Western Australian Journ...,432,https://nla.gov.au/nla.news-article640616,"-33.64166667,151.2677778"


As can be seen above, the dataset contains several columns of useful information, ranging from place names, to newspaper articles, to dates and coordinates. For the purposes of this demonstration, however, we are only interested in the (lat, lon) coordinates and start date of each bushfire.

We proceed by dropping all columns other than the 'coordinates' and 'datestart' column.

In [3]:
df.drop(df.columns.difference(['coordinates', 'datestart']), axis=1, inplace=True)
df.head()

,datestart,coordinates
0,1824-05-14,"-33.90666667,151.0594444"
1,1831-12-24,"-33.75152778,150.9941667"
2,1834-03-15,"-32.055085,115.7459"
3,1835-02-14,"-19.225,138.35"
4,1835-12-19,"-33.64166667,151.2677778"


<h2>Step 2: Preprocessing</h2>

<h3>Project Coordinates</h3>

Hot spot analysis involves applying arithmetic operations to the input coordinates. However, such operations only make sense when dealing with linear coordinate systems. Since our dataset uses an angular coordinate system (WGS84), we must first project the coordinates to some linear system.

There are many types of linear projection systems, and the best one is highly contextual. For the purposes of this demo, we will use the [Map Grid of Australia 1994 (MGA94)](https://www.icsm.gov.au/datum/geocentric-datum-australia-1994-gda94#:~:text=The%20standard%20map%20projection%20associated,Universal%20Transverse%20Mercator%20Grid%20system.) projection system, which is available as part of GeoJikuu's preprocessing.projection package.

The following block of code shows how this projection can be done:

In [4]:
mga1994_projector = MGA1994Projector("WGS84")
wgs84_coordinates = []
for row in df["coordinates"].tolist():
    wgs84_coordinates.append((float(row.split(',')[0]), float(row.split(',')[1])))

results = mga1994_projector.project(wgs84_coordinates)
mga1994_coordinates = results["mga1994_coordinates"]
# unit_conversion = results["unit_conversion"]

df['mga94'] = mga1994_coordinates
df.head()

,datestart,coordinates,mga94
0,1824-05-14,"-33.90666667,151.0594444","(6240766.353818081, 875409.8550096203)"
1,1831-12-24,"-33.75152778,150.9941667","(6258220.583029894, 870038.6276743908)"
2,1834-03-15,"-32.055085,115.7459","(5988838.712066973, -2511909.2671285663)"
3,1835-02-14,"-19.225,138.35","(7851487.897910243, -411913.177344884)"
4,1835-12-19,"-33.64166667,151.2677778","(6269396.349533128, 895909.5787525391)"


<h3>Convert Dates to Time Steps</h3>

Now, we must also convert the dates to time steps. This can be done using the DateConvertor class from GeoJikuu's preprocessing.conversion_tools package. 

In [5]:
date_convertor = DateConvertor("%Y-%m-%d", "%d/%m/%Y")

dates = df["datestart"].tolist()
time_steps = []

for date in dates:
    time_steps.append(date_convertor.date_to_days(date))
    
df['time_step'] = pd.Series(time_steps)
df.head()

,datestart,coordinates,mga94,time_step
0,1824-05-14,"-33.90666667,151.0594444","(6240766.353818081, 875409.8550096203)",665970
1,1831-12-24,"-33.75152778,150.9941667","(6258220.583029894, 870038.6276743908)",668750
2,1834-03-15,"-32.055085,115.7459","(5988838.712066973, -2511909.2671285663)",669562
3,1835-02-14,"-19.225,138.35","(7851487.897910243, -411913.177344884)",669898
4,1835-12-19,"-33.64166667,151.2677778","(6269396.349533128, 895909.5787525391)",670206


Now that we have the projected coordinates and the converted time steps, we can then drop the WGS84 coordinate column (i.e., 'coordinates') and the date column (i.e., 'datestart').

In [6]:
df.drop(df.columns.difference(['mga94', 'time_step']), axis=1, inplace=True)
df.head()

,mga94,time_step
0,"(6240766.353818081, 875409.8550096203)",665970
1,"(6258220.583029894, 870038.6276743908)",668750
2,"(5988838.712066973, -2511909.2671285663)",669562
3,"(7851487.897910243, -411913.177344884)",669898
4,"(6269396.349533128, 895909.5787525391)",670206


<h2>Step 3: Aggregate Data</h2>

To perform a spacetime hot spot analysis on a large number of dispersed points, it typically makes more sense to first aggregate them into clusters. These clusters will then be analysed by the spacetime hot spot algorithm to determine whether any spatiotemporal hot spots (or cold spots) exist among them. Points can be aggregated in many ways, but in this case, we will use a spatiotemporal implementation of the k-Nearest Neighbours algorithm to partition the points and then aggregate by count. We can do this using GeoJikuu's STKNearestNeighbours class which is located in the aggregation.point_aggregators package.


In [7]:
st_knn_aggregator = STKNearestNeighbours(df, "mga94", "time_step")

bushfires_agg = st_knn_aggregator.aggregate(k=4, aggregate_type="mean")

bushfires_agg.head()

Aggregated 7454 points into 369 clusters.


,time_step,midpoint,count
,,,
0,686621.390805,"(6244132.861482037, 886834.035526545, 686621.3...",261
1,687680.444444,"(6252848.654551683, 868507.2994186162, 687680....",27
2,686796.942857,"(6002083.046044371, -2508044.1040300406, 68679...",70
3,687598.934579,"(7897147.706563479, 103549.709538894, 687598.9...",107
4,682174.875000,"(6285727.397096511, 812279.0365990701, 682174....",8


The above output shows that the 7454 bushfires were aggregated into 369 bushfire clusters. 

<h2>Step 4: Spacetime Hot Spot Analysis</h2>

We are finally ready to perform the analysis. This can be done using GeoJikuu's STGiStarHotSpotAnalysis class which is imported from the hypothesis_testing module. It determines which aggregated locations are statistically significant hot spots (or cold spots). We are running the analysis on the 'count' variable, which means we are looking for the clusters where bushfires occur more or less frequently (through space and time) than would be expected given the other clusters in the research area.

In [8]:
sthsa = STGiStarHotSpotAnalysis(bushfires_agg, "midpoint")
results = sthsa.run("count")

Spacetime Getis-Ord Gi* Hot Spot Analysis Summary
-------------------------------------------------
Statistically Significant Clusters: 3
    Statistically Significant Hot Spots: 2
    Statistically Significant Cold Spots: 1
Non-Statistically Significant Clusters: 366
Total Clusters: 369

Null Hypothesis (H₀): The observed pattern of the variable 'count' in cluster i is the result of spatiotemporal randomness alone.
Alpha Level (α): 0.05

Verdict: Sufficient evidence to reject H₀ when α = 0.05 for clusters i = {67, 218, 254}


The above output shows that the analysis found two statistically significant hot spots and one statistically significant cold spot. The dataset IDs of these statistically significant clusters are 67, 218 and 254.

<h3>Convert Data to Initial Format</h3>

Before viewing and mapping our statistically significant hot spots, we first need to convert them from MGA 1994 back to WGS84 (lat, lon) coordinates. This can be done using the inverse_project() function of the MGA1994 Projector class.

Furthermore, we need to convert the corresponding time steps back to dates. This can be done using the DateConvertor class' days_to_date() function.

In [9]:
# Project coordinates back to WGS84 & convert time steps back to dates
string_list = results.midpoint.values.tolist()
tuple_list = []
time_steps = []
for string in string_list:
    x_string = string.strip('(').strip(')').split(", ")
    x = tuple([float(i) for i in x_string])
    tuple_list.append(x)
    time_steps.append(x[2])

results["coordinates"] = mga1994_projector.inverse_project(tuple_list)

dates = []
for time_step in time_steps:
    dates.append(date_convertor.days_to_date(time_step))
    
results['date'] = pd.Series(dates)
results.drop("midpoint", axis=1, inplace=True)

<h3>View Results</h3>

We can view our statistically significant hot and cold spots by filtering the results DataFrame:

In [10]:
sig_results = results[results['significant'] == "TRUE"].sort_values(by=['type'], ascending=False)
sig_results

,z-score,p-value,significant,type,coordinates,date
218,0.005843,0.041932,TRUE,HOT SPOT,"(-35.37588022209752, 145.75600179241445)",16/08/1888
254,0.006331,0.045427,TRUE,HOT SPOT,"(-35.59833333, 145.8241667)",11/11/1890
67,-0.000440,0.003160,TRUE,COLD SPOT,"(-35.39007591749348, 147.32186684582953)",04/04/1884


<h2>Step 5: Map Results</h2>

Finally, it only makes sense that we should visualise our results. There are many ways to do this using third-party libraries. In this case, we will use Leaflet via a Python package called folium.

Each of the statistically significant hot and cold spots are displayed as red or cold dots, respectively. Non-statistically significant clusters are displayed as grey dots.

Clicking on a dot will show a popup containing the corresponding coordinate, date, and test results.

In [11]:
import folium
from folium.plugins import MarkerCluster

aus_coords = [-25.2744, 133.7751]
demo_map = folium.Map(location = aus_coords, zoom_start = 5)

sig_results_hot = sig_results[sig_results['type'] == "HOT SPOT"]
coord_list_hot = sig_results_hot["coordinates"].values.tolist()
date_list_hot = sig_results_hot["date"].values.tolist()
z_score_list_hot = sig_results_hot["z-score"].values.tolist()
p_value_list_hot = sig_results_hot["p-value"].values.tolist()

sig_results_cold = sig_results[sig_results['type'] == "COLD SPOT"]
coord_list_cold = sig_results_cold["coordinates"].values.tolist()
date_list_cold = sig_results_cold["date"].values.tolist()
z_score_list_cold = sig_results_cold["z-score"].values.tolist()
p_value_list_cold = sig_results_cold["p-value"].values.tolist()

insig_results = results[results['significant'] == "FALSE"]
coord_list_insig = insig_results["coordinates"].values.tolist()
date_list_insig = insig_results["date"].values.tolist()
z_score_list_insig = insig_results["z-score"].values.tolist()
p_value_list_insig = insig_results["p-value"].values.tolist()

In [12]:
for i in range(0,len((coord_list_hot))):
    popup = "<b>Coords: </b>" + str(coord_list_hot[i]) + "<br><b>Date: </b>" + str(date_list_hot[i]) + "<br><b>Z-Score: </b> " + str(z_score_list_hot[i]) + "\n" + "<br><b>p-value: </b> " + str(p_value_list_hot[i])
    folium.Circle(coord_list_hot[i], color='red', fillColor='#f03', fillOpacity=0.5, radius=10000, popup=popup).add_to(demo_map)
    
for i in range(0,len((coord_list_cold))):
    popup = "<b>Coords: </b>" + str(coord_list_cold[i]) + "<br><b>Date: </b>" + str(date_list_cold[i]) + "<br><b>Z-Score: </b> " + str(z_score_list_cold[i]) + "\n" + "<br><b>p-value: </b> " + str(p_value_list_cold[i])
    folium.Circle(coord_list_cold[i], color='blue', fillColor='#13a8e8', fillOpacity=0.5, radius=10000, popup=popup).add_to(demo_map)
    
for i in range(0,len((coord_list_insig))):
    popup = "<b>Coords: </b>" + str(coord_list_insig[i]) + "<br><b>Date: </b>" + str(date_list_insig[i]) + "<br><b>Z-Score: </b> " + str(z_score_list_insig[i]) + "\n" + "<br><b>p-value: </b> " + str(p_value_list_insig[i])
    folium.Circle(coord_list_insig[i], color='gray', fillColor='#808080', fillOpacity=0.5, radius=10000, popup=popup).add_to(demo_map)

In [13]:
folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr = 'Esri',
    name = 'Esri Satellite',
    overlay = False,
    control = True
    ).add_to(demo_map)

In [14]:
demo_map

As a final note, the analysis results can be saved as a CSV file by calling the DataFrame's to_csv() function:

In [15]:
results.to_csv('bushfire_sthsa_results.csv')